# 1. Patch descriptors register in 3D

## 1.1 load and pachify depth and rgb in 14*14 size patches

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
from torchvision.io import read_image
from PIL import Image

scene_id = 1
frame_id = 0
## 10* use :06d to turn to 6 digit - but also 10* to turn back just use int as str(int(scene_id)) 
scene_id = f"{scene_id:06d}"
frame_id = f"{frame_id:06d}"



rgb_img = np.array(Image.open(f"datasets/bop23_challenge/datasets/icbin/test/{scene_id}/rgb/{frame_id}.png"))
# depth image with pixel values as float (16 bit)
depth_img = np.array(Image.open(f"datasets/bop23_challenge/datasets/icbin/test/{scene_id}/depth/{frame_id}.png"), dtype="float32")

In [62]:
import torch.nn.functional as F
## To divide image into patches using patchify(image, patch_shape, step) - input as numpy image of (W,H,3)
## step in in pat defines the distance between one patch and the next one (vertically and horizontally). If step ≥ patch_height there is no overlap between patches in the same row. If step ≥ patch_width there is no overlap between patches in the same column.
## step=patch_size to make sure there is no overlap
# First, ensure the image dimensions are divisible by the patch size
# Ensure the image dimensions are divisible by the patch size
def pad_to_multiple(array, multiple):
    height, width, channels = array.shape
    pad_height = (multiple - height % multiple) % multiple
    pad_width = (multiple - width % multiple) % multiple
    padding = [(0, pad_height), (0, pad_width), (0, 0)]  # padd to the b
    return np.pad(array, padding, mode='constant')

#? Pad 0 to depth image might be wrong- should be infinite instead
patch_size = 14 
padded_rgb_img = pad_to_multiple(rgb_img, patch_size) #(490, 644, 3)
padded_depth_img = pad_to_multiple(np.expand_dims(depth_img, axis=-1), patch_size).squeeze() #(490, 644)

In [63]:
from patchify import patchify
rgb_patches = patchify(padded_rgb_img, (14,14,3), step=patch_size) # patch shape [14,14,3]
depth_patches = patchify(padded_depth_img, (14,14), step=patch_size) # patch shape [14,14]
# ## Merge the patches to 1 image again using unpatchify
# reconstructed_image = unpatchify(patches, image.shape)

In [64]:
rgb_patches.shape # (35, 46, 1, 14, 14, 3) ## means that we have create 35*46*1 = 1610 patches - 10* 1 is for the channel here only 1 patch for the channel
depth_patches.shape # (35, 46, 14, 14) ## means that we have create 35*46 = 1610 patches

(35, 46, 14, 14)

## 1.2 Filter invalid patches
Use the mask images to filter out these patches - if its center doesn't belong to the mask then is excluded
so we divide the rbg and masks in the same way then can just check the validity within the patches

In [70]:
# Load the masks - we have mutiple visible masks for the input iamge- sum it up (clip it in range of 0,1)- then u have the mask for the entire scene
# count the number of objects in the scene/frame via scene_gt_info.json

import json
scene_gt = json.load(open(f"datasets/bop23_challenge/datasets/icbin/test/{scene_id}/scene_gt_info.json", "r"))
num_objs = len(scene_gt[str(int(frame_id))])

masks = list()
for obj_id in range(num_objs):
    mask = np.array(Image.open(f"datasets/bop23_challenge/datasets/icbin/test/{scene_id}/mask_visib/{frame_id}_{obj_id:06d}.png"))
    masks.append(mask)

entire_mask = np.clip(np.sum(np.array(masks), axis=0), 0,1)

In [77]:
# Pad and patchify the mask
padded_mask = pad_to_multiple(np.expand_dims(entire_mask, axis=-1), patch_size).squeeze()
mask_patches = patchify(padded_mask, (14,14), step=patch_size) # patch shape [14,14]
mask_patches.shape

(35, 46, 14, 14)

In [89]:
# Fallen the patches for easier iteration
rgb_patches = rgb_patches.reshape(-1, patch_size, patch_size, 3)
mask_patches = mask_patches.reshape(-1, patch_size, patch_size)
depth_patches = depth_patches.reshape(-1, patch_size, patch_size)

valid_rgb_patches = []
valid_mask_patches = []
valid_depth_patches = []

## Center of an image is height/2, width/2 and here is (7,7)
for rgb_patch, mask_patch, depth_patch in zip(rgb_patches, mask_patches, depth_patches):
    center_y, center_x = patch_size // 2, patch_size // 2
    if mask_patch[center_y, center_x] == 1:  # Check if the center belongs to the foreground
        valid_rgb_patches.append(rgb_patch)
        valid_mask_patches.append(mask_patch)
        valid_depth_patches.append(depth_patch)

In [104]:
# Permute to tensor with shaape (num_patches, 3, 14, 14)
valid_rgb_patches = torch.tensor(valid_rgb_patches).permute(0,3,1,2)

/tmp/ipykernel_1307237/1390511641.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  valid_rgb_patches = torch.tensor(valid_rgb_patches).permute(0,3,1,2)


AttributeError: 'list' object has no attribute 'shape'

In [135]:
valid_depth_patches = torch.tensor(np.array(valid_depth_patches))
valid_mask_patches = torch.tensor(np.array(valid_mask_patches, dtype="uint8"))

## 1.3 Load features via Dinov2

In [91]:
# load config file
import hydra
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose

# Initialize Hydra and compose the configuration
initialize(config_path="configs")
cfg = compose(config_name="run_inference")
OmegaConf.set_struct(cfg, False)

/tmp/ipykernel_1307237/531409554.py:7: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="configs")


In [92]:
from hydra.utils import instantiate
# define the cnos model
cfg_segmentor = cfg.model.segmentor_model
if "fast_sam" in cfg_segmentor._target_:
    logging.info("Using FastSAM, ignore stability_score_thresh!")
else:
    cfg.model.segmentor_model.stability_score_thresh = 0.97 # for sam
cfg.model.descriptor_model.model.num_block = 19

model = instantiate(cfg.model).to("cuda")

Using cache found in /home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main
/home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


In [93]:
model.descriptor_model.model.num_block

19

In [193]:
import torchvision.transforms as T
import torch.nn.functional as F

def patches_feature_extraction(valid_rgb_patches, cnos_model):
    # crop_rgb: numpy array
    temps = np.transpose(np.array(valid_rgb_patches), (0,2,3,1))
    rgb_normalize = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]
    )
    normalized_patches = torch.stack([rgb_normalize(patch) for patch in temps])
    with torch.no_grad(): 
        feature_patches= cnos_model.descriptor_model.model(normalized_patches.to("cuda"))
    return feature_patches

In [194]:
rgb_patch_features = patches_feature_extraction(valid_rgb_patches, model)

In [195]:
rgb_patch_features.shape

torch.Size([309, 1024])

# 2. Bag-of-words descriptors